# GenJam + Evaluate Function

### Define Measure and Phrase Classes

In [1]:
import random

class Measure:
    events: int # Array of 8 consisting events
    fitness: int # The fitness of the measure
        
    def __init__(self, events=False):
        if events==False:
            note_sample = list(range(1, 15)) + [0]*5 + [15]*15 # Give rest and longer notes more chance to be chosed
            self.events = [random.choice(note_sample) for _ in range(8)] # Randmoly allocate events
            self.fitness = 0
            pass
        
        else:
            self.events = events
            self.fitness = 0
            pass
    
    def evaluate(self, fitness):
        self.fitness = fitness
        pass
    
class Phrase:
    measures: int # Array of 4 consisting measures
    fitness: int # The fitness of the measure
        
    def __init__(self, measure_population, measures=False):
        if measures==False:
            self.measures = random.choices(measure_population, k=4) # Select 4 measures randomly allowed possible repetitions
            self.fitness = sum([self.measures[i].fitness for i in range(4)]) # Naive fitness alocation
            pass
        
        else:
            self.measures = measures
            self.fitness = sum([self.measures[i].fitness for i in range(4)]) # Naive fitness alocation
            # Do not repeat best measure
            if self.measures[0]==self.measures[1] or self.measures[1]==self.measures[2] or self.measures[2]==self.measures[3]:
                self.fitness = -10
            pass
    
    def evaluate(self, fitness):
        self.fitness = fitness
        pass
    

### Define Measure and Phrase Crossovers

In [2]:
def measure_crossover(parent1, parent2):
    crossover_point = random.randint(1, 7)
    offspring1 = Measure(parent1.events[:crossover_point] + parent2.events[crossover_point:])
    offspring2 = Measure(parent2.events[:crossover_point] + parent1.events[crossover_point:])

    return offspring1, offspring2

def phrase_crossover(parent1, parent2):
    crossover_point = random.randint(1, 3)
    offspring1 = Phrase(parent1.measures[:crossover_point] + parent2.measures[crossover_point:])
    offspring2 = Phrase(parent2.measures[:crossover_point] + parent1.measures[crossover_point:])

    return offspring1, offspring2

### Define Measure Mutations

In [3]:
class Measure(Measure):
    
    def mu_transpose(self):
        scale = random.randint(-5, 5)
        self.events = [min(max((e+scale),1),14) if e!=0 and e!=15 else e for e in self.events]
        pass
    
    def mu_reverse(self):
        self.events.reverse()
        pass
    
    def mu_rotate(self):
        scale = random.randint(2, 5)
        self.events = self.events[scale:] + self.events[:scale]
        pass
    
    def mu_invert(self):
        self.events = [15-e for e in self.events]
        pass
    
    def mu_sortup(self):
        new_event_sorted = sorted([e for e in self.events if e!=0 and e!=15])
        self.events = [new_event_sorted.pop(0) if e!=0 and e!=15 else e for e in self.events]
        pass
    
    def mu_sortdown(self):
        new_event_sorted = sorted([e for e in self.events if e!=0 and e!=15], reverse=True)
        self.events = [new_event_sorted.pop(0) if e!=0 and e!=15 else e for e in self.events]
        pass
    
    def mu_invert_reverse(self):
        self.mu_invert()
        self.mu_reverse()
        pass

### Define Phrase Mutations

In [4]:
import heapq

class Phrase(Phrase):
    
    def mu_reverse(self):
        self.measures.reverse()
        pass
    
    def mu_rotate(self):
        scale = random.randint(1, 3)
        self.measures = self.measures[scale:] + self.measures[:scale]
        pass
    
    def mu_sequence(self):
        index = random.randint(0, 3)  # Choose a replace index
        sequence = index + random.choice([-1, 1])  # Choose a sequence index
        if sequence==-1: sequence=1 # Only one choice for 0 and 3 indexes
        elif sequence==4: sequence=2
        self.measures[index] = self.measures[sequence] 
        pass
    
    def mu_repair(self, population):
        measure_population = population[0]
        random_measure = random.choices(measure_population)[0]
        least_fitness = min(range(4), key=lambda i: self.measures[i].fitness)
        self.measures[least_fitness] = random_measure
        pass
    
    def mu_super(self, population):
        measure_population = population[0]
        three_tournament = [random.choices(measure_population, k=3) for _ in range(4)]
        selected_super = [max(three_tournament[i], key=lambda x: x.fitness) for i in range(4)]
        random.shuffle(selected_super)
        self.measures = selected_super # Discard the child
        pass
    
    def mu_thinlick(self, population):
        measure_population = population[0]
        phrase_population = population[1]
        measure_counts = {}
        for measure in self.measures: # Set measures in this phrase
            if measure not in measure_counts:
                    measure_counts[measure] = 0
                    
        for phrase in phrase_population: # Count measures in population
            for measure in phrase.measures:
                if measure in measure_counts:
                    measure_counts[measure] += 1

        lick_index = self.measures.index(max(measure_counts, key=measure_counts.get)) # Obtain the first index the lick is appeared
        self.measures[lick_index] = random.choices(measure_population)[0]
        pass
    
    def mu_orphan(self, population):
        measure_population = population[0]
        phrase_population = population[1]
        measure_counts = {}
        for measure in measure_population: # Set measures in measure population
            if measure not in measure_counts:
                    measure_counts[measure] = 0
                    
        for phrase in phrase_population: # Count measures in population
            for measure in phrase.measures:
                if measure in measure_counts:
                    measure_counts[measure] += 1
                    
        self.measures = heapq.nsmallest(4, measure_counts, key=measure_counts.get) # Discard the child
        random.shuffle(self.measures) # Shuffle the measures randomly
                    
        

In [5]:
# Test phrase mutation based on the example in the report

def strp(phrase, measure_population):
    measure_index = [measure_population.index(m) for m in phrase.measures]
    return measure_index


# Create measure_population with 20 and phrase_population with 10 population
measure_population = [Measure() for _ in range(64)]
phrase_population = [Phrase(measure_population) for _ in range(10)] + [Phrase(measure_population, [measure_population[57], measure_population[57], measure_population[11], measure_population[38]])]



### Add random mutation function to Measure and Phrase

In [6]:
class Measure(Measure):
    def __init__(self, events=False):
        self.mutations = [
                self.mu_transpose,
                self.mu_reverse,
                self.mu_rotate,
                self.mu_invert,
                self.mu_sortup,
                self.mu_sortdown,
                self.mu_invert_reverse
            ]
        if events==False:
            self.events = [random.randint(0, 15) for _ in range(8)] # Randmoly allocate events
            self.fitness = 0
            pass
        
        else:
            self.events = events
            self.fitness = 0
            pass
        
    def apply_random_mutation(self):
        random_mutation = random.choice(self.mutations)
        random_mutation()
        pass
    
class Phrase(Phrase): 
    def __init__(self, measure_population, measures=False):
        self.mutations = [
                self.mu_reverse,
                self.mu_rotate,
                self.mu_sequence,
                self.mu_repair,
                self.mu_super,
                self.mu_thinlick,
                self.mu_orphan
            ]
        if measures==False:
            self.measures = random.choices(measure_population, k=4) # Select 4 measures randomly allowed possible repetitions
            self.fitness = sum([self.measures[i].fitness for i in range(4)]) # Naive fitness alocation
            pass
        
        else:
            self.measures = measures
            self.fitness = 0
            pass

    def apply_random_mutation(self, phrase_population, measure_population):
        random_mutation = random.choice(self.mutations)
        
        # Apply the random function with different input sizes
        if random_mutation.__code__.co_argcount > 1: random_mutation([measure_population, phrase_population])
        else: random_mutation()
        pass

### Define GenJam

In [7]:
class GenJam:
    def __init__(self, num_measures = 64, num_phrases = 48, num_measure_tournament = 4, num_phrase_tournament = 4):
        self.num_measures = num_measures
        self.num_phrases = num_phrases
        self.num_measure_tournament = num_measure_tournament
        self.num_phrase_tournament = num_phrase_tournament
        self.measure_population = [Measure() for _ in range(num_measures)]
        self.phrase_population = [Phrase(measure_population) for _ in range(num_phrases)]
        pass
    
    def generate(self, num_generations = 1):
        for iteration in range(num_generations):
            self.select_generate_measure()
            self.select_generate_phrase()
            print("An Iteration is Passed. The Best Achived Phrase: ")
            self.phrase_population.sort(key=lambda x: x.fitness)
            self.print_phrase(self.phrase_population[0])            
        pass
    
    def select_generate_measure(self):
        for i in range(self.num_measures//4):
            
            # Selection process
            p1, p2 = self.tournament_selection(self.measure_population, self.num_measure_tournament)
            
            # Crossover process
            o1, o2 = measure_crossover(p1, p2)
            
            # Mutation process
            random.choice([o1, o2]).apply_random_mutation()
            
            # Evaluation Process
            o1.fitness = self.print_measure_fit(o1)
            o2.fitness = self.print_measure_fit(o2)
            
            # Replace peocess
            self.measure_population.sort(key=lambda x: x.fitness)
            self.measure_population[:2] = [o1, o2]        
        pass
    
    def select_generate_phrase(self):
        for i in range(self.num_phrases//4):
            
            # Selection process
            p1, p2 = self.tournament_selection(self.phrase_population, self.num_phrase_tournament)
            
            # Crossover process
            o1, o2 = phrase_crossover(p1, p2)
            
            # Mutation process
            random.choice([o1, o2]).apply_random_mutation(self.phrase_population, self.measure_population)
            
            # Evaluation Process
            o1.fitness = self.print_phrase_fit(o1)
            o2.fitness = self.print_phrase_fit(o2)
            
            # Replace peocess
            self.phrase_population.sort(key=lambda x: x.fitness)
            self.phrase_population[:2] = [o1, o2]
        pass
    
    def tournament_selection(self, population, k):
        selection = random.sample(population, k)
        parents = sorted(selection, key=lambda x: x.fitness, reverse=True)[:2]
        return parents[0], parents[1]
    
    def print_measure_fit(self, m):
        return int(input(str(m.events)+", Evaluated Fitness for this Measure: "))
        
    def print_phrase_fit(self, p):
        measures_events = [m.events for m in p.measures]
        return int(input(str(measures_events)+", Evaluated Fitness for this Phrase: "))
    
    def print_phrase(self, p):
        measures_events = [m.events for m in p.measures]
        print(measures_events)
        pass

### Define Fitness Function

In [8]:
from collections import Counter

class Fitness:
    
    def __init__(self, w_large_intervals = 2, max_interval = 7, \
                 w_downbeat = 2, w_halfbar = 1, w_long_notes = 1,
                 w_pattern_matching = 3, w_repeat = 2, \
                 max_ninterval = 4, len_pattern = 3, max_repeat = 2):

        self.w_large_intervals = w_large_intervals
        self.max_interval = max_interval
        self.w_downbeat = w_downbeat
        self.w_halfbar = w_halfbar
        self.w_long_notes = w_long_notes
        self.w_repeat = w_repeat
        self.max_ninterval = max_ninterval
        self.w_pattern_matching = w_pattern_matching
        self.len_pattern = len_pattern
        self.max_repeat = max_repeat
        pass
    
    def evaluate(self, improv):
        score = - (self.e_large_intervals(self.max_interval, improv) * self.w_large_intervals) \
                + (self.e_downbeat(improv) * self.w_downbeat) \
                + (self.e_halfbar(improv) * self.w_halfbar) \
                - (self.e_long_notes(self.max_ninterval, improv) * self.w_long_notes) \
                + (self.e_pattern_matching(self.len_pattern, improv) * self.w_pattern_matching) \
                - (self.e_repeat(self.max_repeat, improv) * self.w_repeat)
        
        return score
    
    def e_large_intervals(self, max_interval, improv):
        miss = 0
        notes = [e for e in improv if e not in [0, 15]] # Only consider intervals
        for i in range(len(notes)-1):
            miss += abs(notes[i+1]-notes[i]) > max_interval
        return miss
    
    def e_downbeat(self, improv):
        if improv[0]!=15 or improv[0]!=0: return 1
        return 0
    
    def e_halfbar(self, improv):
        if improv[4]!=15 or improv[4]!=0: return 1
        return 0
    
    def e_long_notes(self, max_ninterval, improv):
        miss = sum(1 for i in range(len(improv) - max_ninterval) if \
                     len(set(improv[i:i + max_ninterval - 1])) == 1 and improv[i]==15)
        return miss
    
    def e_pattern_matching(self, len_pattern, improv):
        hit = 0
        for i in range(len(improv) - len_pattern + 1):
            pattern = tuple(improv[i:i+len_pattern])
            hit += Counter(tuple(improv[j:j+len_pattern]) for j in range(i+1, len(improv) - len_pattern+1))[pattern]
        return hit
    
    def e_repeat(self, max_repeat, improv):
        notes = [e for e in improv if e not in [15]] # Only consider intervals
        miss = sum(1 for i in range(len(notes) - max_repeat + 1) if \
                     len(set(improv[i:i + max_repeat])) == 1)
        return miss

In [9]:
class Measure(Measure):
    
    def evaluate(self, evaluation):
        self.fitness = evaluation.evaluate(self.events)
        return self.fitness

    
class Phrase(Phrase):
    
    def evaluate(self, fitness):
        self.fitness = sum([self.measures[i].fitness for i in range(4)]) 
        return self.fitness
    

In [10]:
class GenJam(GenJam):

    def __init__(self, fitness, num_measures = 20, num_phrases = 10, num_measure_tournament = 3, num_phrase_tournament = 3):
        self.fitness = fitness
        self.num_measures = num_measures
        self.num_phrases = num_phrases
        self.num_measure_tournament = num_measure_tournament
        self.num_phrase_tournament = num_phrase_tournament
        self.measure_population = [Measure() for _ in range(num_measures)]
        [m.evaluate(self.fitness) for m in self.measure_population]
        self.phrase_population = [Phrase(measure_population) for _ in range(num_phrases)]
        pass
    
    def print_measure_fit(self, m):
        return m.evaluate(self.fitness)
        
    def print_phrase_fit(self, p):
        return p.evaluate(self.fitness)
    

In [11]:
fitness = Fitness()

In [17]:
jamer = GenJam(fitness)

In [18]:
jamer.generate(25)

An Iteration is Passed. The Best Achived Phrase: 
[[0, 0, 15, 1, 15, 15, 14, 3], [3, 15, 15, 4, 15, 15, 15, 15], [5, 15, 0, 12, 15, 12, 15, 0], [5, 15, 0, 12, 15, 12, 15, 0]]
An Iteration is Passed. The Best Achived Phrase: 
[[15, 3, 15, 15, 15, 15, 15, 15], [15, 0, 12, 14, 7, 10, 3, 15], [2, 13, 0, 15, 15, 15, 14, 12], [2, 13, 0, 15, 15, 15, 14, 12]]
An Iteration is Passed. The Best Achived Phrase: 
[[15, 5, 15, 15, 4, 15, 2, 2], [7, 0, 15, 5, 7, 15, 15, 8], [15, 8, 15, 15, 9, 8, 15, 15], [7, 0, 15, 5, 7, 15, 15, 8]]
An Iteration is Passed. The Best Achived Phrase: 
[[15, 3, 15, 15, 15, 15, 15, 15], [15, 8, 15, 15, 9, 8, 15, 15], [15, 5, 14, 8, 8, 2, 1, 15], [15, 3, 15, 15, 15, 15, 15, 15]]
An Iteration is Passed. The Best Achived Phrase: 
[[15, 15, 2, 15, 15, 15, 0, 15], [15, 2, 5, 0, 5, 15, 13, 0], [8, 15, 0, 5, 5, 15, 0, 1], [12, 15, 0, 15, 15, 6, 15, 0]]
An Iteration is Passed. The Best Achived Phrase: 
[[3, 7, 1, 7, 4, 10, 3, 5], [0, 15, 3, 0, 15, 15, 14, 0], [15, 5, 15, 15, 4, 1

In [27]:
jamer = GenJam(fitness)

In [28]:
jamer.generate(25)

An Iteration is Passed. The Best Achived Phrase: 
[[15, 15, 0, 15, 15, 15, 12, 15], [15, 15, 0, 15, 15, 15, 12, 15], [12, 15, 0, 15, 15, 6, 15, 0], [15, 5, 3, 15, 15, 0, 8, 15]]
An Iteration is Passed. The Best Achived Phrase: 
[[15, 13, 15, 0, 0, 15, 1, 15], [0, 10, 15, 15, 0, 0, 10, 0], [15, 13, 15, 0, 0, 15, 1, 15], [15, 13, 15, 0, 0, 15, 1, 15]]
An Iteration is Passed. The Best Achived Phrase: 
[[15, 1, 0, 13, 7, 4, 14, 14], [15, 1, 0, 13, 7, 4, 14, 14], [3, 15, 15, 10, 3, 0, 15, 1], [15, 1, 0, 13, 7, 4, 14, 14]]
An Iteration is Passed. The Best Achived Phrase: 
[[0, 0, 15, 9, 7, 3, 11, 3], [15, 15, 1, 15, 9, 14, 15, 0], [15, 8, 15, 15, 9, 8, 15, 15], [0, 10, 15, 15, 0, 0, 10, 0]]
An Iteration is Passed. The Best Achived Phrase: 
[[3, 15, 15, 10, 3, 0, 15, 1], [15, 1, 0, 13, 7, 4, 14, 14], [4, 15, 0, 15, 0, 15, 15, 6], [3, 15, 15, 4, 15, 15, 15, 15]]
An Iteration is Passed. The Best Achived Phrase: 
[[15, 15, 15, 0, 11, 6, 15, 7], [12, 6, 10, 9, 8, 1, 11, 4], [15, 15, 15, 0, 11, 6,